# part0: imports

In [9]:
import os, sys, pathlib
from pprint import pprint 
from importlib import reload
import logging
from typing import Callable
import warnings



import pandas as pd
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
import scipy.linalg as linalg

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import MaxNLocator


import pyaldata as pyal

%matplotlib inline

try:
    nbPath = pathlib.Path.cwd()
    RepoPath = nbPath.parent
    os.chdir(RepoPath)

    from tools import utilityTools as utility
    from tools import dataTools as dt
    import params
    defs = params.monkey_defs
    
    set_rc =  params.set_rc_params
    root = params.root

finally:
    os.chdir(nbPath)

#### Print the summary of all the datassets

check out the results [here](https://github.com/AtMostafa/notebook/blob/main/2021-cca-project/dataset-summary.md).

In [10]:
_AllAnimalList = ['Chewie', 'Chewie2', 'Mihili', 'Jaco', "Chips", "MrT", 'Han', 'Lando']

_AllAnimalFiles=[]
for animal in _AllAnimalList:
    _AllAnimalFiles.extend(utility.find_file(root / animal, 'mat'))

# for i,fname in enumerate(_AllAnimalFiles):
#     _df = pyal.mat2dataframe(fname, shift_idx_fields=True)
#     print(i)
#     dt.summary(_df)

Categories I want to separate:

- sessions with 2 areas, similar number of neurons each, as many monkeys as possible!
- 3 sessions for each area (M1, PMd, S1), most number of neurons

In [11]:
_AllAnimalFiles= [path.split(os.sep)[-1] for path in _AllAnimalFiles]

In [4]:
# GoodDataList = {'dualArea':{}, 'M1':{}, 'PMd':{}, 'S1':{}}

# #------------------------------------

# GoodDataList['dualArea']['Chewie'] = [_AllAnimalFiles[13],
#                                       _AllAnimalFiles[12],
#                                       _AllAnimalFiles[3],
#                                       _AllAnimalFiles[1],
#                                       _AllAnimalFiles[10]
#                                      ]

# GoodDataList['dualArea']['Mihili'] = [_AllAnimalFiles[64],
#                                       _AllAnimalFiles[57],
#                                       _AllAnimalFiles[68],
#                                       _AllAnimalFiles[69]
#                                      ]

# GoodDataList['dualArea']['MrT'] = [_AllAnimalFiles[105],
#                                    _AllAnimalFiles[103]
#                                   ]

# #-----------------------------------

# GoodDataList['M1']['Chewie'] = [_AllAnimalFiles[13],
#                                 _AllAnimalFiles[10],
#                                 _AllAnimalFiles[12],
#                                 _AllAnimalFiles[1]
#                                ]

# GoodDataList['M1']['Chewie2'] = [_AllAnimalFiles[28],
#                                  _AllAnimalFiles[22],
#                                  _AllAnimalFiles[36],
#                                  _AllAnimalFiles[47]
#                                 ]

# GoodDataList['M1']['Mihili'] = [_AllAnimalFiles[64],
#                                 _AllAnimalFiles[57],
#                                 _AllAnimalFiles[68]
#                                ]

# GoodDataList['M1']['Jaco'] = [_AllAnimalFiles[85],
#                               _AllAnimalFiles[87],
#                               _AllAnimalFiles[81]
#                              ]

# #-----------------------------------

# GoodDataList['PMd']['Chewie'] = [_AllAnimalFiles[9],
#                                 _AllAnimalFiles[13],
#                                 _AllAnimalFiles[8],
#                                 _AllAnimalFiles[11]
#                                ]

# GoodDataList['PMd']['Mihili'] = [_AllAnimalFiles[56],
#                                  _AllAnimalFiles[74],
#                                  _AllAnimalFiles[68]
#                                 ]

# GoodDataList['PMd']['MrT'] = [_AllAnimalFiles[103],
#                               _AllAnimalFiles[105],
#                               _AllAnimalFiles[102]
#                              ]


# #------------------------------------

# GoodDataList['S1']['Chips'] = [_AllAnimalFiles[95],
#                                _AllAnimalFiles[97],
#                                _AllAnimalFiles[94],
#                                _AllAnimalFiles[92]
#                               ]

# GoodDataList['S1']['Han'] = [_AllAnimalFiles[119],
#                              _AllAnimalFiles[107],
#                              _AllAnimalFiles[116],
#                              _AllAnimalFiles[115]
#                             ]

# GoodDataList['S1']['Lando'] = [_AllAnimalFiles[120],
#                              _AllAnimalFiles[121],
#                              _AllAnimalFiles[122]
#                             ]


# print(f'the variable `GoodDataList` contains the session names')

the variable `GoodDataList` contains the session names


The code above relies on the host PC having the exact fiels as my laptop.  
So I replace the values below for reliability.

In [12]:
GoodDataList = {'dualArea':{}, 'M1':{}, 'PMd':{}, 'S1':{}}

#------------------------------------

GoodDataList['dualArea']['Chewie'] = ['Chewie_CO_VR_2016-09-14.mat',
                                      'Chewie_CO_CS_2016-10-21.mat',
                                      'Chewie_CO_FF_2016-10-05.mat',
                                      'Chewie_CO_CS_2016-10-14.mat',
                                      'Chewie_CO_FF_2016-10-13.mat'
                                     ]

GoodDataList['dualArea']['Mihili'] = ['Mihili_CO_VR_2014-03-06.mat',
                                      'Mihili_CO_VR_2014-03-03.mat',
                                      'Mihili_CO_FF_2014-02-17.mat',
                                      'Mihili_CO_VR_2014-03-04.mat'
                                     ]

GoodDataList['dualArea']['MrT'] = ['MrT_CO_VR_2013-09-09.mat',
                                   'MrT_CO_VR_2013-09-05.mat'
                                  ]

#-----------------------------------

GoodDataList['M1']['Chewie'] = ['Chewie_CO_VR_2016-09-14.mat',
                                'Chewie_CO_FF_2016-10-13.mat',
                                'Chewie_CO_CS_2016-10-21.mat',
                                'Chewie_CO_CS_2016-10-14.mat'
                               ]

GoodDataList['M1']['Chewie2'] = ['Chewie_CO_CS_2015-03-19.mat',
                                 'Chewie_CO_CS_2015-03-13.mat',
                                 'Chewie_CO_CS_2015-03-11.mat',
                                 'Chewie_CO_CS_2015-03-12.mat'
                                ]

GoodDataList['M1']['Mihili'] = ['Mihili_CO_VR_2014-03-06.mat',
                                'Mihili_CO_VR_2014-03-03.mat',
                                'Mihili_CO_FF_2014-02-17.mat'
                               ]

GoodDataList['M1']['Jaco'] = ['Jaco_CO_CS_2016-01-28.mat',
                              'Jaco_CO_CS_2016-02-04.mat',
                              'Jaco_CO_CS_2016-02-17.mat'
                             ]

#-----------------------------------

GoodDataList['PMd']['Chewie'] = ['Chewie_CO_FF_2016-09-21.mat',
                                 'Chewie_CO_VR_2016-09-14.mat',
                                 'Chewie_CO_FF_2016-09-15.mat',
                                 'Chewie_CO_FF_2016-09-19.mat'
                                ]

GoodDataList['PMd']['Mihili'] = ['Mihili_CO_FF_2014-02-18.mat',
                                 'Mihili_CO_CS_2014-09-29.mat',
                                 'Mihili_CO_FF_2014-02-17.mat'
                                ]

GoodDataList['PMd']['MrT'] = ['MrT_CO_VR_2013-09-05.mat',
                              'MrT_CO_VR_2013-09-09.mat',
                              'MrT_CO_FF_2013-08-21.mat'
                             ]

#-----------------------------------

GoodDataList['S1']['Chips'] = ['Chips_20151204_TD_nosort_notrack_noemg.mat',
                               'Chips_20151201_TD_nosort_notrack_noemg.mat',
                               'Chips_20151211_TD_nosort_notrack_noemg.mat',
                               'Chips_20170505_TD_nosort_notrack_noemg.mat'
                              ]

GoodDataList['S1']['Han'] = ['Han_COactpas_2017-10-31.mat',
                             'Han_COactpas_2017-12-04.mat',
                             'Han_COactpas_2017-10-24.mat',
                             'Han_COactpas_2017-11-28.mat'
                            ]

GoodDataList['S1']['Lando'] = ['Lando_20170731_TD_nosort_notrack_noemg.mat',
                               'Lando_20170917_TD_nosort_notrack_noemg.mat',
                               'Lando_20170803_TD_nosort_notrack_noemg.mat'
                              ]


Adding the aggregate of *M1* and *PMd* as **MCx**

In [13]:
MCx = {}
for area in ['M1', 'PMd', 'dualArea']:
    for animal in GoodDataList[area]:
        if animal not in MCx:
            MCx[animal] = []
        MCx[animal].extend(GoodDataList[area][animal])
        MCx[animal] = list(set(MCx[animal]))

GoodDataList['MCx'] = MCx

In [14]:
SingleSessionEx = {'Chewie2':['Chewie_CO_CS_2015-03-12.mat'],
                   'Mihili':['Mihili_CO_FF_2014-02-17.mat'],
                   'Jaco':['Jaco_CO_CS_2016-02-17.mat']
                   }

In [22]:
# warnings.filterwarnings("ignore")
# rows = []

# good_files = []
# for animal in GoodDataList['MCx'].keys():
#     for file in GoodDataList['MCx'][animal]:
#         good_files.append(str(root) + f'/{animal}/{file}')

# for file in good_files:
#     raw_df = pyal.mat2dataframe(file, shift_idx_fields=True)
#     df = defs.prep_general(raw_df)

#     dic = {
#         'animal': file.split(os.sep)[-2],
#         'file': file.split(os.sep)[-1],
#         'areas': [x for x in list(raw_df.columns) if '_spikes' in x],
#         'trials_all': len(raw_df),
#         'trials_left': len(df),
#         'neurons_all': sum([raw_df[x][0].shape[1] for x in list(raw_df.columns) if '_spikes' in x]),
#         'neurons_left': df['MCx_spikes'][0].shape[1] 
#     }
#     rows.append(dic)

# summary = pd.DataFrame(rows)
# summary

,animal,file,areas,trials_all,trials_left,neurons_all,neurons_left
0,Chewie,Chewie_CO_FF_2016-09-21.mat,"[M1_spikes, PMd_spikes]",874,176,303,289
1,Chewie,Chewie_CO_VR_2016-09-14.mat,"[M1_spikes, PMd_spikes]",837,195,353,345
2,Chewie,Chewie_CO_FF_2016-10-05.mat,"[M1_spikes, PMd_spikes]",660,202,244,239
3,Chewie,Chewie_CO_FF_2016-10-13.mat,"[M1_spikes, PMd_spikes]",801,239,223,210
4,Chewie,Chewie_CO_FF_2016-09-15.mat,"[M1_spikes, PMd_spikes]",940,182,309,299
5,Chewie,Chewie_CO_CS_2016-10-21.mat,"[M1_spikes, PMd_spikes]",293,286,295,281
6,Chewie,Chewie_CO_CS_2016-10-14.mat,"[M1_spikes, PMd_spikes]",770,740,278,269
7,Chewie,Chewie_CO_FF_2016-09-19.mat,"[M1_spikes, PMd_spikes]",826,186,295,288
8,Chewie2,Chewie_CO_CS_2015-03-19.mat,[M1_spikes],1081,1026,74,73
9,Chewie2,Chewie_CO_CS_2015-03-13.mat,[M1_spikes],1089,1037,88,87


In [24]:
# summary.groupby('animal')['neurons_left'].agg(['mean', 'sem'])

,mean,sem
animal,,
Chewie,277.500000,14.257830
Chewie2,84.750000,4.090130
Jaco,63.333333,8.838049
Mihili,117.333333,3.756476
MrT,57.666667,1.201850
